In [1]:
import ReadNewickFormat
import BinaryTree
import pandas as pd
import re

alphabet = ["A","T","C","G"]

type_LeafNode = BinaryTree.BinaryTree.LeafNode #for comparing node types

type_InternalNode = BinaryTree.BinaryTree.InternalNode

In [2]:
T, Error =  ReadNewickFormat.run("((2,3),((1,4),5))") #Testeingabe für parser

testSeq = ["A","A","A","A","C"]

if Error:
    print(Error.as_string())
else: print(T)
    


((2,3),((1,4),5))


In [3]:
def prepareLeaves(T,sequences):
    """
    Sorting the leaf array and assigning the sequences.
    """
    leaves = T.getLeaves()
    leaves.sort(key=lambda x: x.num)
    for index, value in enumerate(sequences):
        leaves[index].setSequence(value)
    

In [4]:
def score(node, seq_index, delta = None):
    """
    Score funtion for a node.
    """
    
    if delta is None: 
        delta = lambda x,y: 0 if x == y else 1 #function for smallparsimony
    else:
        df = delta
        delta = lambda x,y: df.loc[x][y] #function for weighted parsiomony
    inf = float('inf')
    res = {} #dictionary for storing score for each alphabet char
    if type(node) == type_LeafNode:
        for value in alphabet:
            if node.sequence[seq_index] == value:
                res[value] = 0
            else:
                res[value] = inf
        node.score = res
    else:
        for value in alphabet:
            res[value] = min([node.left.score[i] + delta(i,value) for i in alphabet]) + min([node.right.score[i] + delta(i,value) for i in alphabet])
        
        node.score = res
            
    



In [5]:
def smallParsimony(T, leaves,delta = None):
    n = len (leaves[0].sequence) #length of sequences

    res = 0
    for i in range(n):
        ripeNodes = [] #ripeNodes have two tagged children
        tag = True 

        for j in range(len(leaves)):
            leaves[j].setTag(tag) #tag children
            score(leaves[j], i,delta)
            parent = leaves[j].parent

            if parent.left.tag == tag and parent.right.tag == tag: #check if internalNode has two tags
                ripeNodes.append(parent)

        while ripeNodes: #while there a ripeNodes, calculate the scores for each ripe node
            node = ripeNodes.pop()
            node.setTag(tag)
            score(node,i,delta)
            parent = node.parent

            if parent == None: #if node is root
                break

            if parent.left.tag == tag and parent.right.tag == tag: #if parent is ripe
                ripeNodes.append(parent)

        res += min(T.root.score.values()) 
        tag = not tag #set new tag for next round

    return res

In [6]:
def readFile(filename):
    """
    Reads a text file.
    """
    text = []
    with open(filename) as file:
        for line in file:
            if ((sline := line.rstrip()) == ""): #if there is an empty line
                continue
            text.append(sline)
    return text

In [7]:
def getCostMatrix(costMatrix):
    """
    Gets cost matrix from string list.
    """
    #costMatrix = [str.replace(s, " ","") for s in costMatrix] #delete all whitespaces #!!TODO: Fehler bei mehrstelligen Zahlen
    
    costMatrix = [re.split(' +', s) for s in costMatrix] #split string into list on white spaces
    
    cr_name = [c for c in costMatrix[0]] 

    df = pd.DataFrame(columns=cr_name, index=cr_name)

    for i in range(0,len(cr_name)):
        for j in range(0,i+1):
            cost = float(costMatrix[i+1][j+1])
            df.iloc[i,j] = cost
            df.iloc[j,i] = cost
    return df

In [8]:
sequences = readFile("sequenzen.txt")
try:
    costMatrix = getCostMatrix(readFile("kosten.txt"))
except e:
    costMatrix = pd.DataFrame()

T, Error =  ReadNewickFormat.run(readFile("topologie.txt")[0])


if Error:
    print(Error.as_string())
elif not all([len(sequences[0]) == len(seq) for seq in sequences]):
    print("Sequenzen haben nicht gleiche Länge.")
elif len(sequences) != len(T.getLeaves()):
    print("Mehr Sequenzen als Blätter im Newick-Format")
elif costMatrix.empty:
    print("Falsches Matrixformat")
else:
    prepareLeaves(T, sequences)
    print(f'Weighted small parsimony score: {smallParsimony(T, T.getLeaves(),costMatrix)}')

Weighted small parsimony score: 30.0
